### This is the file to merge the train2id.txt and train2id_type.txt

In [18]:
from tqdm.notebook import tqdm

with open('./data/3_openKE/synthesize/test2id.txt', 'r') as file1:
    next(file1)
    lines1 = file1.readlines()

with open('./data/3_openKE/synthesize/test2id_label.txt', 'r') as file2:
    next(file2)
    labels = [line.strip().split(' ')[1] for line in tqdm(file2.readlines()[1:])]

merged_lines = [f"{line1.strip()} {label}" for line1, label in tqdm(zip(lines1, labels))]

  0%|          | 0/4289981 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [19]:
print(merged_lines[:5])

['14291 8575 14 benign', '16352 8575 14 benign', '13722 8575 12 benign', '9054 17168 4 benign', '24342 8575 1 benign']


In [20]:
# 寫入新的txt文件
with open('./data/3_openKE/synthesize/merged_test.txt', 'w') as merged_file:
    merged_file.write('\n'.join(merged_lines))
    print("output file at './data/3_openKE/synthesize/merged_test.txt'")

output file at './data/3_openKE/synthesize/merged_test.txt'


### This is to merge all the 3 files together

In [32]:
from tqdm.notebook import tqdm

# 設定三個文件的路徑
file_paths = [
    './data/3_openKE/synthesize/merged_train.txt',
    './data/3_openKE/synthesize/merged_test.txt',
    './data/3_openKE/synthesize/merged_valid.txt'
]

# 初始化合併的結果
merged_lines = []

# 遍歷每個文件
for file_path in file_paths:
    with open(file_path, 'r') as file:
        # 使用tqdm追蹤讀取進度
        lines = tqdm(file, desc=f"Reading {file_path}", leave=False)
        
        # 將每一行添加到合併結果中
        merged_lines.extend(lines)

Reading ./data/3_openKE/synthesize/merged_train.txt: 0it [00:00, ?it/s]

Reading ./data/3_openKE/synthesize/merged_test.txt: 0it [00:00, ?it/s]

Reading ./data/3_openKE/synthesize/merged_valid.txt: 0it [00:00, ?it/s]

In [33]:
print(len(merged_lines))

51479771


In [38]:
merged_lines[:5]

['7042 8575 1 benign\n',
 '7042 8575 1 benign\n',
 '7042 8575 1 benign\n',
 '7042 8575 1 benign\n',
 '7042 8575 1 benign\n']

In [34]:
with open('./data/3_openKE/synthesize/merged_all.txt', 'w') as merged_file:
    # 使用tqdm追蹤寫入進度
    for line in tqdm(merged_lines, desc="Writing merged file", leave=False):
        merged_file.write(line)

print("Output file at './data/3_openKE/synthesize/merged_all.txt'")

Writing merged file:   0%|          | 0/51479771 [00:00<?, ?it/s]

Output file at './data/3_openKE/synthesize/merged_all.txt'


In [42]:
def process_and_save_graphs_dfs(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Create a graph and a reverse graph
    graph = {}
    reverse_graph = {}

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 4:
            continue  # Skip lines that don't have exactly 4 parts

        source, destination, _ = parts[:3]

        if source not in graph:
            graph[source] = set()
        graph[source].add(destination)

        if destination not in reverse_graph:
            reverse_graph[destination] = set()
        reverse_graph[destination].add(source)

    # Find connected components using DFS
    visited = set()
    components = []

    for node in set(graph.keys()).union(set(reverse_graph.keys())):  # Include all nodes
        if node not in visited:
            component = set()
            dfs(graph, node, visited, component)

            # Include nodes that are only in reverse_graph
            additional_nodes = set()
            for rev_node in component:
                if rev_node in reverse_graph:
                    for source in reverse_graph[rev_node]:
                        if source not in component:
                            additional_nodes.add(source)

            component.update(additional_nodes)
            components.append(component)

    # Group lines based on components
    grouped_lines = {i: [] for i in range(len(components))}
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 4:
            continue

        source, destination, _, _ = parts
        for i, component in enumerate(components):
            if source in component or destination in component:
                grouped_lines[i].append(line)
                break

    # Save each component to a separate file
    # 在腳本中保存文件的部分
    output_folder = '/workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs'
    for i, lines in grouped_lines.items():
        with open(f'{output_folder}graph{i + 1}.txt', 'w') as file:
            file.writelines(lines)


    return len(grouped_lines)


INPUT_FILE_PATH = '/workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/merged_all.txt'
process_and_save_graphs_dfs(INPUT_FILE_PATH)


3

In [43]:
from tqdm import tqdm

def dfs(graph, start_node, visited, component):
    visited.add(start_node)
    component.add(start_node)
    if start_node in graph:
        for neighbor in graph[start_node]:
            if neighbor not in visited:
                dfs(graph, neighbor, visited, component)

def process_and_save_graphs_dfs(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    graph = {}
    reverse_graph = {}
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 4:
            continue
        source, destination, _ = parts[:3]
        if source not in graph:
            graph[source] = set()
        graph[source].add(destination)
        if destination not in reverse_graph:
            reverse_graph[destination] = set()
        reverse_graph[destination].add(source)

    visited = set()
    components = []

    for node in tqdm(set(graph.keys()).union(set(reverse_graph.keys())), desc="Processing Nodes"):
        if node not in visited:
            component = set()
            dfs(graph, node, visited, component)
            additional_nodes = set()
            for rev_node in component:
                if rev_node in reverse_graph:
                    additional_nodes.update(reverse_graph[rev_node] - component)
            component.update(additional_nodes)
            components.append(component)

    grouped_lines = {i: [] for i in range(len(components))}
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 4:
            continue
        source, destination, _, _ = parts
        for i, component in enumerate(components):
            if source in component or destination in component:
                grouped_lines[i].append(line)

    output_folder = '/workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/'
    for i, lines in tqdm(grouped_lines.items(), desc="Saving Files"):
        output_path = f'{output_folder}graph{i + 1}.txt'
        with open(output_path, 'w') as file:
            file.writelines(lines)
        print(f"File saved: {output_path}")

    return len(grouped_lines)

INPUT_FILE_PATH = '/workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/merged_all.txt'
num_files = process_and_save_graphs_dfs(INPUT_FILE_PATH)
print(f"Total number of files created: {num_files}")



Processing Nodes: 100%|██████████| 26868/26868 [00:00<00:00, 176382.53it/s]

Saving Files:   0%|          | 1/219 [00:01<05:55,  1.63s/it]

File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph1.txt



Saving Files:   1%|          | 2/219 [00:01<03:02,  1.19it/s]

File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph2.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph3.txt



Saving Files:   2%|▏         | 4/219 [00:09<09:54,  2.77s/it]

...

File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph159.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph160.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph161.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph162.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph163.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph164.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph165.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph166.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph167.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph168.txt
File saved


Saving Files:  82%|████████▏ | 180/219 [00:20<00:00, 47.72it/s]

File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph174.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph175.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph176.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph177.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph178.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph179.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph180.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph181.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph182.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph183.txt
File saved


Saving Files: 100%|██████████| 219/219 [00:20<00:00, 10.64it/s]

File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph201.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph202.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph203.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph204.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph205.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph206.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph207.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph208.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph209.txt
File saved: /workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/graph210.txt
File saved

#### For statistics of the triplets num

In [45]:
import os
from tqdm.notebook import tqdm

def remove_duplicates_in_folder(folder_path):
    for filename in tqdm(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                lines = file.readlines()
            
            unique_lines = set(lines)
            with open(file_path, 'w') as file:
                file.writelines(unique_lines)

output_folder = '/workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs/'  # 更換為您的輸出文件夾路徑
remove_duplicates_in_folder(output_folder)


  0%|          | 0/220 [00:00<?, ?it/s]

In [51]:
import os

def count_rows_in_files(folder_path):
    row_counts = {}
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                lines = file.readlines()
            
            row_counts[filename] = len(lines)
    
    return row_counts

# output_folder = '/path/to/your/output/folder'  # 更換為您的輸出文件夾路徑
row_counts = count_rows_in_files(output_folder)

print("Filename: Count")
for filename, count in row_counts.items():
    print(f"{filename}: {count}")


Filename: Count
group_0.txt: 492598
graph1.txt: 4659
graph2.txt: 44054
graph3.txt: 1352
graph4.txt: 492446
graph5.txt: 4847
graph6.txt: 3875
graph7.txt: 4847
graph8.txt: 28208
graph9.txt: 3302
graph10.txt: 383
graph11.txt: 2555
graph12.txt: 3302
graph13.txt: 743
graph14.txt: 3875
graph15.txt: 3603
graph16.txt: 4847
graph17.txt: 441
graph18.txt: 3875
graph19.txt: 490946
graph20.txt: 8
graph21.txt: 8
graph22.txt: 3
graph23.txt: 9
graph24.txt: 26
graph25.txt: 121
graph26.txt: 5
graph27.txt: 1471
graph28.txt: 26
graph29.txt: 14
graph30.txt: 3
graph31.txt: 97
graph32.txt: 3
graph33.txt: 1
graph34.txt: 10
graph35.txt: 11
graph36.txt: 1
graph37.txt: 3
graph38.txt: 9
graph39.txt: 3
graph40.txt: 53
graph41.txt: 2
graph42.txt: 26
graph43.txt: 3
graph44.txt: 10
graph45.txt: 5
graph46.txt: 9
graph47.txt: 166
graph48.txt: 467
graph49.txt: 135
graph50.txt: 4
graph51.txt: 948
graph52.txt: 2545
graph53.txt: 3
graph54.txt: 1
graph55.txt: 8
graph56.txt: 1045
graph57.txt: 10
graph58.txt: 3
graph59.txt: 2

In [49]:
import os

def count_files_by_triplets(folder_path):
    triplets_count = {}
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                lines = file.readlines()
            
            count = len(lines)
            triplets_count[count] = triplets_count.get(count, 0) + 1
    
    return triplets_count

def sort_and_print_triplets_count(triplets_count):
    # 按照Triplets数量从大到小排序
    sorted_triplets_count = sorted(triplets_count.items(), key=lambda x: x[0], reverse=True)

    for count, num_files in sorted_triplets_count:
        print(f"Triplets Number {count}: {num_files} files")

# 示例路径，根据实际情况替换
# output_folder = '/path/to/your/output/folder'
files_count_by_triplets = count_files_by_triplets(output_folder)
sort_and_print_triplets_count(files_count_by_triplets)


Triplets Number 492598: 1 files
Triplets Number 492446: 1 files
Triplets Number 490946: 1 files
Triplets Number 44054: 1 files
Triplets Number 28208: 1 files
Triplets Number 4847: 3 files
Triplets Number 4659: 1 files
Triplets Number 3875: 3 files
Triplets Number 3603: 1 files
Triplets Number 3302: 2 files
Triplets Number 2555: 1 files
Triplets Number 2545: 1 files
Triplets Number 2251: 1 files
Triplets Number 1495: 1 files
Triplets Number 1490: 1 files
Triplets Number 1473: 1 files
Triplets Number 1472: 1 files
Triplets Number 1471: 2 files
Triplets Number 1430: 1 files
Triplets Number 1414: 1 files
Triplets Number 1368: 1 files
Triplets Number 1358: 1 files
Triplets Number 1356: 1 files
Triplets Number 1352: 1 files
Triplets Number 1271: 1 files
Triplets Number 1248: 1 files
Triplets Number 1096: 1 files
Triplets Number 1092: 1 files
Triplets Number 1061: 1 files
Triplets Number 1058: 1 files
Triplets Number 1045: 1 files
Triplets Number 948: 1 files
Triplets Number 824: 1 files
Trip